# Bài 2

In [1]:
from google.colab import drive
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import numpy as np

drive.mount('/content/drive')

drive_zip_path = '/content/drive/My Drive/VinaFood21.zip'

!unzip -q -o "{drive_zip_path}" -d ./data

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

DATA_DIR = './data/VinaFood21'
TRAIN_DIR = os.path.join(DATA_DIR, 'train')
TEST_DIR = os.path.join(DATA_DIR, 'test')

Mounted at /content/drive


In [2]:
input_size = 224

data_transforms = {
    'train': transforms.Compose([
        transforms.Lambda(lambda img: img.convert("RGB")),
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Lambda(lambda img: img.convert("RGB")),
        transforms.Resize(input_size + 32),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

image_datasets = {
    'train': datasets.ImageFolder(TRAIN_DIR, data_transforms['train']),
    'test': datasets.ImageFolder(TEST_DIR, data_transforms['test'])
}

dataloaders = {
    'train': DataLoader(image_datasets['train'], batch_size=128, shuffle=True, num_workers=2),
    'test': DataLoader(image_datasets['test'], batch_size=128, shuffle=False, num_workers=2)
}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'test']}
class_names = image_datasets['train'].classes

print(f"Số lớp (NUM_CLASSES): {len(class_names)}")
print(f"Kích thước tập huấn luyện: {dataset_sizes['train']}")
print(f"Kích thước tập test: {dataset_sizes['test']}")

Số lớp (NUM_CLASSES): 21
Kích thước tập huấn luyện: 10044
Kích thước tập test: 6682


In [3]:
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, **kwargs):
        super(ConvBlock, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, **kwargs)

        self.bn = nn.BatchNorm2d(out_channels)

        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        return self.relu(self.bn(self.conv(x)))

class Inception(nn.Module):
    def __init__(self, in_channels, ch1x1, ch3x3red, ch3x3, ch5x5red, ch5x5, pool_proj):
        super(Inception, self).__init__()

        self.branch1 = ConvBlock(in_channels, ch1x1, kernel_size=1)

        self.branch2 = nn.Sequential(
            ConvBlock(in_channels, ch3x3red, kernel_size=1),
            ConvBlock(ch3x3red, ch3x3, kernel_size=3, padding=1)
        )

        self.branch3 = nn.Sequential(
            ConvBlock(in_channels, ch5x5red, kernel_size=1),
            ConvBlock(ch5x5red, ch5x5, kernel_size=5, padding=2)
        )

        self.branch4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1, ceil_mode=True),
            ConvBlock(in_channels, pool_proj, kernel_size=1)
        )

    def forward(self, x):
        branch1 = self.branch1(x)
        branch2 = self.branch2(x)
        branch3 = self.branch3(x)
        branch4 = self.branch4(x)

        return torch.cat([branch1, branch2, branch3, branch4], 1)

class GoogLeNet(nn.Module):
    def __init__(self, num_classes=1000):
        super(GoogLeNet, self).__init__()

        self.conv1 = ConvBlock(in_channels=3, out_channels=64, kernel_size=7, stride=2, padding=3)

        self.maxpool1 = nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True)

        self.conv2_1 = ConvBlock(64, 64, kernel_size=1)
        self.conv2_2 = ConvBlock(64, 192, kernel_size=3, padding=1)

        self.maxpool2 = nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True)

        self.inception3a = Inception(192, 64, 96, 128, 16, 32, 32)
        self.inception3b = Inception(256, 128, 128, 192, 32, 96, 64)

        self.maxpool3 = nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True)

        self.inception4a = Inception(480, 192, 96, 208, 16, 48, 64)
        self.inception4b = Inception(512, 160, 112, 224, 24, 64, 64)
        self.inception4c = Inception(512, 128, 128, 256, 24, 64, 64)
        self.inception4d = Inception(512, 112, 144, 288, 32, 64, 64)
        self.inception4e = Inception(528, 256, 160, 320, 32, 128, 128)

        self.maxpool4 = nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True)

        self.inception5a = Inception(832, 256, 160, 320, 32, 128, 128)
        self.inception5b = Inception(832, 384, 192, 384, 48, 128, 128)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.dropout = nn.Dropout(p=0.4)
        self.fc = nn.Linear(1024, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2_1(x)
        x = self.conv2_2(x)
        x = self.maxpool2(x)

        x = self.inception3a(x)
        x = self.inception3b(x)
        x = self.maxpool3(x)

        x = self.inception4a(x)
        x = self.inception4b(x)
        x = self.inception4c(x)
        x = self.inception4d(x)
        x = self.inception4e(x)
        x = self.maxpool4(x)

        x = self.inception5a(x)
        x = self.inception5b(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.dropout(x)
        x = self.fc(x)

        return x

In [4]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="PIL")

model = GoogLeNet(num_classes=21).to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

for epoch in range(25):
    model.train()
    running_loss = 0.0

    for inputs, labels in dataloaders['train']:
        inputs = inputs.to(DEVICE)
        labels = labels.to(DEVICE)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / dataset_sizes['train']
    print(f'Epoch [{epoch+1}/25], Loss: {epoch_loss:.4f}')

    model.eval()
    all_labels = []
    all_preds = []

    with torch.no_grad():
        for inputs, labels in dataloaders['test']:
            inputs = inputs.to(DEVICE)
            labels = labels.to(DEVICE)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())

    scheduler.step()

    print(f"Learning Rate hiện tại: {scheduler.get_last_lr()[0]:.10f}")


accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds, average='macro', zero_division=0)
recall = recall_score(all_labels, all_preds, average='macro', zero_division=0)
f1 = f1_score(all_labels, all_preds, average='macro', zero_division=0)

print(f"Độ chính xác (Accuracy): {accuracy * 100:.2f}%")
print(f"Precision (Macro): {precision:.4f}")
print(f"Recall (Macro): {recall:.4f}")
print(f"F1-score (Macro): {f1:.4f}")

Epoch [1/25], Loss: 2.5568
Learning Rate hiện tại: 0.0001000000
Epoch [2/25], Loss: 2.1667
Learning Rate hiện tại: 0.0001000000
Epoch [3/25], Loss: 2.0046
Learning Rate hiện tại: 0.0001000000
Epoch [4/25], Loss: 1.8942
Learning Rate hiện tại: 0.0001000000
Epoch [5/25], Loss: 1.7941
Learning Rate hiện tại: 0.0000500000
Epoch [6/25], Loss: 1.6551
Learning Rate hiện tại: 0.0000500000
Epoch [7/25], Loss: 1.5994
Learning Rate hiện tại: 0.0000500000
Epoch [8/25], Loss: 1.5214
Learning Rate hiện tại: 0.0000500000
Epoch [9/25], Loss: 1.4840
Learning Rate hiện tại: 0.0000500000
Epoch [10/25], Loss: 1.4436
Learning Rate hiện tại: 0.0000250000
Epoch [11/25], Loss: 1.3520
Learning Rate hiện tại: 0.0000250000
Epoch [12/25], Loss: 1.3158
Learning Rate hiện tại: 0.0000250000
Epoch [13/25], Loss: 1.2851
Learning Rate hiện tại: 0.0000250000
Epoch [14/25], Loss: 1.2669
Learning Rate hiện tại: 0.0000250000
Epoch [15/25], Loss: 1.2357
Learning Rate hiện tại: 0.0000125000
Epoch [16/25], Loss: 1.1907
Learni